In [1]:
from nltk import word_tokenize
import re
import time
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import MWETokenizer
from nltk.corpus import wordnet
wnl = WordNetLemmatizer()

D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### pdfplumber、borb、PyMuPDF

In [3]:
pdf_path = "./data/00006_2020.pdf"

In [6]:
def cost_time(func):
    def fun(*args, **kwargs):
        t = time.perf_counter()
        result = func(*args, **kwargs)
        print(f'func {func.__name__} cost time:{time.perf_counter() - t:.2f} s')
        return result
    return fun

#### 1. pdfplumber

In [4]:
import pdfplumber

@cost_time
def test_pdfplumber(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page.extract_text()
    
test_pdfplumber(pdf_path)

func test_pdfplumber cost time:43.85 s


#### 2. borb

In [5]:
import typing
from borb.pdf import Document
from borb.pdf import PDF
from borb.toolkit import SimpleTextExtraction

@cost_time
def test_borb(pdf_path):

    # read the Document
    doc: typing.Optional[Document] = None
    l: SimpleTextExtraction = SimpleTextExtraction()
    with open(pdf_path, "rb") as in_file_handle:
        doc = PDF.loads(in_file_handle, [l])

    # check whether we have read a Document
    assert doc is not None

    # print the text on the first Page
    print(l.get_text()[0])

test_borb(pdf_path)

P r s et
Ass owe
(Stock Code : 6)
i d
Hol
m it ngs Li ed
dom • K i n g Austra i t e d lia
U n & Sa • • lt h f e t H
o e a y n H g
K o n
g
• M
a li t y i n b i
l ia a el n T
d R e c h C
n h i n o l
a o g
y
•
N e
w Z
e l a
a n
d
• T
h i a
l C a l
n e a d n
E n e r
gy •
a C n a
d a
• e N
h t l r e a
n a c i r e s d m A
f o s e t a t S d e t i n U •
A al ort 20 20
nnu Rep
Gearing up
for Sustainable Business
Annual Report 2020
func test_borb cost time:538.03 s


Exception in thread Thread-5:
Traceback (most recent call last):
  File "D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\urllib3\connectionpool.py", line 700, in urlopen
    self._prepare_proxy(conn)
  File "D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\urllib3\connectionpool.py", line 994, in _prepare_proxy
    conn.connect()
  File "D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\urllib3\connection.py", line 364, in connect
    self.sock = conn = self._connect_tls_proxy(hostname, conn)
  File "D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\urllib3\connection.py", line 499, in _connect_tls_proxy
    socket = ssl_wrap_socket(
  File "D:\Programming\Python\Environment_Construction\Anaconda\lib\site-packages\urllib3\util\ssl_.py", line 453, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls)
  File "D:\Programming\Python\Environment_Construction\Anacon

#### 3. PyMuPDF

In [17]:
import fitz

@cost_time
def test_PyMuPDF(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        page.get_text("text")
    
        
toc = test_PyMuPDF(pdf_path)

func test_PyMuPDF cost time:0.74 s


结论：综上，使用pymupdf(其下的fitz工具包)效率最高

In [9]:
# 用pymupdf提取页面文本（可以解决排版问题，这个包会自动识别版面，但是不确定这个功能的稳定性）
def get_pages(pdf_path):
    doc = fitz.open(pdf_path)
    return doc

doc = get_pages(pdf_path)
doc[57].get_text("text")  # P58

In [12]:
# 用pymupdf提取目录
temp_path = "./data/00005_2021.pdf"
def get_toc(pdf_path):
    doc = fitz.open(pdf_path)
    toc = doc.get_toc()
    return toc

toc = get_toc(temp_path)
toc

[[1, 'HSBC Holdings plc Environmental, Social and Governance Update 2019 ', 1],
 [1, 'Contents', 3],
 [1, 'Overview', 4],
 [2, 'HSBC at a glance ', 4],
 [2, 'Group Chief Executive’s review ', 6],
 [2, 'Delivering our strategy ', 8],
 [2, 'Our approach ', 10],
 [1, 'Customers', 12],
 [2, 'Our Retail Banking and Wealth Management customers', 14],
 [2, 'Our Commercial Banking customers', 18],
 [2, 'Our Global Banking and Markets customers', 20],
 [2, 'Our Global Private Banking customers', 22],
 [2, 'Customer developments across all global businesses', 24],
 [1, 'Employees', 28],
 [2, 'Listening to our people', 30],
 [2, 'Diversity and inclusion', 32],
 [2, 'Well-being and developing our people', 34],
 [1, 'Supporting sustainable growth', 36],
 [2, 'Sustainable  finance', 38],
 [2, 'Our approach to sustainability risk management  ', 44],
 [2, 'Suppliers', 45],
 [2, 'Sustainable operations', 46],
 [2, 'Committed to communities', 48],
 [1, 'A responsible business culture ', 50],
 [2, 'Ensur

### nltk分词 vs re正则匹配

In [22]:
@cost_time
def test_nltk(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        t = page.get_text("text")
        tokenizer = MWETokenizer([('climate', 'change')], separator = ' ')
        wordlist = tokenizer.tokenize(nltk.word_tokenize(t))
test_nltk(pdf_path)

func test_nltk cost time:1.00 s


In [29]:
@cost_time
def test_re(pdf_path):
    doc = fitz.open(pdf_path)
    pattern = re.compile("climate change")
    for page in doc:
        t = page.get_text("text")
        pattern.search(t)
test_re(pdf_path)

func test_re cost time:0.72 s


结论：两者效率差不多，用nltk